# Fusion des bases de données

Dans ce notebook, on fusionne toutes nos bases de données pour créer notre base de données de travail.

In [1]:
import numpy as np
import pandas as pd

In [2]:
#On importe le notebook dans lequel on a récupéré et nettoyé nos données
%run Recuperation_donnees.ipynb

## Fusion des données et création de la base

On commence par ajouter une variable avec la population et une variable avec le PIB par habitant, pour l'année correspondante, à la base des médailles.

In [3]:
#Fonction qui, pour une année donnée, fait une jointure entre la base des médailles et la population et le PIB par habitant 
#pour cette année donnée

def fonction_jointure(annee):
    
    df_medailles_jointure = df_medailles_copy[df_medailles_copy['Annee'] == annee]
    
    variable_annee_pop = str(annee) + " [YR" + str(annee) + "]"
    df_pop_jointure = df_pop[['Country Code', variable_annee_pop]]
    
    variable_annee_pib = str(annee) + " [YR" + str(annee) + "]"
    df_pib_jointure = df_pib_hab[['Country Code', variable_annee_pib]]
    
    variable_annee_sport = str(annee)
    df_invest_sport_jointure = df_invest_sport_merged[['Country Code', variable_annee_sport]]
    
    df_merged1 = df_medailles_jointure.merge(df_pop_jointure, on="Country Code")
    df_merged1.rename({variable_annee_pop: "Population"}, axis=1, inplace=True)
    
    df_merged2 = df_merged1.merge(df_pib_jointure, on="Country Code")
    df_merged2.rename({variable_annee_pib: "PIB par hab"}, axis=1, inplace=True)
    
    #Pour l'investissement dans la sport, comme il y a peu de données, on fait une jointure à gauche
    df_merged = df_merged2.merge(df_invest_sport_jointure, how="left", on="Country Code")
    df_merged.rename({variable_annee_sport: "Investissement sport"}, axis=1, inplace=True)
    
    return(df_merged)   

In [4]:
#on teste la fonction
fonction_jointure(1992).head(10)

,Country Name,Country Code,Gold,Silver,Bronze,Total,Annee,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,Population,PIB par hab,Investissement sport
0,United States,USA,37,34,37,108,1992,Barcelona,Spain,ESP,USA,2.565140e+08,25418.990776,19250.647
1,People's Republic of China,CHN,16,22,16,54,1992,Barcelona,Spain,ESP,CHN,1.164970e+09,366.460692,NaN
2,Cuba,CUB,14,6,11,31,1992,Barcelona,Spain,ESP,CUB,1.073639e+07,2057.103595,NaN
3,Spain,ESP,13,7,2,22,1992,Barcelona,Spain,ESP,ESP,3.915768e+07,16112.188915,NaN
4,Republic of Korea,KOR,12,5,12,29,1992,Barcelona,Spain,ESP,KOR,4.374796e+07,8126.670390,NaN
5,Hungary,HUN,11,12,7,30,1992,Barcelona,Spain,ESP,HUN,1.036934e+07,3735.105820,NaN
6,France,FRA,8,5,16,29,1992,Barcelona,Spain,ESP,FRA,5.885122e+07,23813.712246,NaN
7,Australia,AUS,7,9,11,27,1992,Barcelona,Spain,ESP,AUS,1.749500e+07,18604.188270,NaN
8,Canada,CAN,7,4,7,18,1992,Barcelona,Spain,ESP,CAN,2.837126e+07,20879.848330,NaN
9,Italy,ITA,6,5,8,19,1992,Barcelona,Spain,ESP,ITA,5.679709e+07,23243.474528,NaN


In [5]:
#On construit la base dans laquelle on a la population et le PIB par habitant pour l'année en cours

df = pd.DataFrame(columns=[])

for annee in df_medailles_copy['Annee'].unique():
    df_merged = fonction_jointure(annee)
    df = pd.concat([df, df_merged])

df

,Country Name,Country Code,Gold,Silver,Bronze,Total,Annee,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,Population,PIB par hab,Investissement sport
0,United States,USA,37,34,37,108,1992,Barcelona,Spain,ESP,USA,2.565140e+08,25418.990776,19250.647
1,People's Republic of China,CHN,16,22,16,54,1992,Barcelona,Spain,ESP,CHN,1.164970e+09,366.460692,NaN
2,Cuba,CUB,14,6,11,31,1992,Barcelona,Spain,ESP,CUB,1.073639e+07,2057.103595,NaN
3,Spain,ESP,13,7,2,22,1992,Barcelona,Spain,ESP,ESP,3.915768e+07,16112.188915,NaN
4,Republic of Korea,KOR,12,5,12,29,1992,Barcelona,Spain,ESP,KOR,4.374796e+07,8126.670390,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Finland,FIN,0,0,2,2,2020,Tokyo,Japan,JPN,FIN,5.530719e+06,48773.281169,NaN
60,Côte d'Ivoire,CIV,0,0,1,1,2020,Tokyo,Japan,JPN,CIV,2.637828e+07,2325.723705,NaN
61,Ghana,GHA,0,0,1,1,2020,Tokyo,Japan,JPN,GHA,3.107294e+07,2205.529016,NaN
62,Republic of Moldova,MDA,0,0,1,1,2020,Tokyo,Japan,JPN,MDA,2.620495e+06,4547.059721,1733.200


On ajoute une variable indicatrice indiquant si le pays est le pays hôte ou pas.

In [6]:
df['Pays_hote_oui_non'] = (df['Country Code'] == df['Code_pays_hote'])
df['Pays_hote_oui_non'] = df['Pays_hote_oui_non'].astype(int)
df

,Country Name,Country Code,Gold,Silver,Bronze,Total,Annee,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,Population,PIB par hab,Investissement sport,Pays_hote_oui_non
0,United States,USA,37,34,37,108,1992,Barcelona,Spain,ESP,USA,2.565140e+08,25418.990776,19250.647,0
1,People's Republic of China,CHN,16,22,16,54,1992,Barcelona,Spain,ESP,CHN,1.164970e+09,366.460692,NaN,0
2,Cuba,CUB,14,6,11,31,1992,Barcelona,Spain,ESP,CUB,1.073639e+07,2057.103595,NaN,0
3,Spain,ESP,13,7,2,22,1992,Barcelona,Spain,ESP,ESP,3.915768e+07,16112.188915,NaN,1
4,Republic of Korea,KOR,12,5,12,29,1992,Barcelona,Spain,ESP,KOR,4.374796e+07,8126.670390,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Finland,FIN,0,0,2,2,2020,Tokyo,Japan,JPN,FIN,5.530719e+06,48773.281169,NaN,0
60,Côte d'Ivoire,CIV,0,0,1,1,2020,Tokyo,Japan,JPN,CIV,2.637828e+07,2325.723705,NaN,0
61,Ghana,GHA,0,0,1,1,2020,Tokyo,Japan,JPN,GHA,3.107294e+07,2205.529016,NaN,0
62,Republic of Moldova,MDA,0,0,1,1,2020,Tokyo,Japan,JPN,MDA,2.620495e+06,4547.059721,1733.200,0


On crée des variables indicatrices pour l'année.

In [7]:
df_dummies = pd.get_dummies(df, columns=["Annee"])
df_dummies

,Country Name,Country Code,Gold,Silver,Bronze,Total,Lieu,Pays_hote,Code_pays_hote,ISO 3166-1,...,Investissement sport,Pays_hote_oui_non,Annee_1992,Annee_1996,Annee_2000,Annee_2004,Annee_2008,Annee_2012,Annee_2016,Annee_2020
0,United States,USA,37,34,37,108,Barcelona,Spain,ESP,USA,...,19250.647,0,1,0,0,0,0,0,0,0
1,People's Republic of China,CHN,16,22,16,54,Barcelona,Spain,ESP,CHN,...,NaN,0,1,0,0,0,0,0,0,0
2,Cuba,CUB,14,6,11,31,Barcelona,Spain,ESP,CUB,...,NaN,0,1,0,0,0,0,0,0,0
3,Spain,ESP,13,7,2,22,Barcelona,Spain,ESP,ESP,...,NaN,1,1,0,0,0,0,0,0,0
4,Republic of Korea,KOR,12,5,12,29,Barcelona,Spain,ESP,KOR,...,NaN,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Finland,FIN,0,0,2,2,Tokyo,Japan,JPN,FIN,...,NaN,0,0,0,0,0,0,0,0,1
60,Côte d'Ivoire,CIV,0,0,1,1,Tokyo,Japan,JPN,CIV,...,NaN,0,0,0,0,0,0,0,0,1
61,Ghana,GHA,0,0,1,1,Tokyo,Japan,JPN,GHA,...,NaN,0,0,0,0,0,0,0,0,1
62,Republic of Moldova,MDA,0,0,1,1,Tokyo,Japan,JPN,MDA,...,1733.200,0,0,0,0,0,0,0,0,1
